In [ ]:
import numpy as np
import pandas as pd
import os

# --- 配置参数 ---
output_filename = 'stress.csv'
# 模拟区域的边长（米）。这应该与 KMC_Simulator 中的 box_size 保持一致，
# 确保应力场覆盖整个 KMC 模拟盒子。
box_size_for_stress_field = 1.0e-3 # 示例：1毫米立方体

# 每个维度上的离散点数量。例如，10 表示 10x10x10 = 1000 个应力点。
# 增加这个值会使应力场更精细，但文件更大，KMC 读取时间可能更长。
num_points_per_dim = 20

# 最大应力迹值 (Pa)。假设我们希望最大拉应力迹达到 400 MPa。
# 注意：MPa (兆帕) 需要转换为 Pa (帕斯卡)。 1 MPa = 1e6 Pa。
max_stress_trace_value = 6000.0e6 # 示例：400 MPa = 400 * 10^6 Pa

print(f"--- Stress Field Generation Configuration ---")
print(f"Output File: {output_filename}")
print(f"Cube Size for Stress Field: {box_size_for_stress_field:.1e} m")
print(f"Points per Dimension: {num_points_per_dim}")
print(f"Total Stress Points: {num_points_per_dim**3}")
print(f"Maximum Stress Trace: {max_stress_trace_value/1e6:.1f} MPa")
print(f"--------------------------------------------")

# --- 生成点并计算应力迹 ---
points_data = []
point_id = 0

# 在每个维度上创建等间距的坐标点
# np.linspace(start, stop, num) 生成从 start 到 stop 包含 num 个点的数组
x_coords = np.linspace(0, box_size_for_stress_field, num_points_per_dim)
y_coords = np.linspace(0, box_size_for_stress_field, num_points_per_dim)
z_coords = np.linspace(0, box_size_for_stress_field, num_points_per_dim)

# 遍历所有可能的 (x, y, z) 组合来生成网格点
for x in x_coords:
    for y in y_coords:
        for z in z_coords:
            # 将当前坐标归一化到 [0, 1] 范围
            # 这样，(0,0,0) 归一化后是 (0,0,0)，(L,L,L) 归一化后是 (1,1,1)
            x_norm = x / box_size_for_stress_field
            y_norm = y / box_size_for_stress_field
            z_norm = z / box_size_for_stress_field

            # 计算梯度因子：
            # 我们希望在 (L,L,L) 角 (归一化后是 (1,1,1)) 处应力最大，
            # 在 (0,0,0) 角 (归一化后是 (0,0,0)) 处应力最小。
            # 采用简单的线性求和，其值范围从 0 (在 (0,0,0)) 到 3 (在 (1,1,1))。
            # 然后除以 3.0 将其归一化到 [0, 1] 范围。
            gradient_factor = (x_norm + y_norm + z_norm) / 3.0

            # 计算当前点的应力迹
            # stress_trace 在 (0,0,0) 处为 0，在 (L,L,L) 处为 max_stress_trace_value
            stress_trace = max_stress_trace_value * gradient_factor

            # 将点数据添加到列表中
            points_data.append([x, y, z, stress_trace])
            point_id += 1 # 确保 ID 从 0 开始连续递增

# --- 创建 Pandas DataFrame 并保存到 CSV ---
df = pd.DataFrame(points_data, columns=['X', 'Y', 'Z', 'Trace'])

# 显示前几行和后几行，以便验证数据和梯度
print("\nGenerated Stress Points (first 5 rows):")
print(df.head())
print("\nGenerated Stress Points (last 5 rows - 应该在 (L,L,L) 附近显示最大应力):")
print(df.tail())

print(f"\nMax Trace in generated data: {df['Trace'].max():.2e} Pa ({df['Trace'].max()/1e6:.2f} MPa)")
print(f"Min Trace in generated data: {df['Trace'].min():.2e} Pa ({df['Trace'].min()/1e6:.2f} MPa)")

# 保存为 CSV 文件
# index=False 避免将 DataFrame 的索引写入 CSV 文件
df.to_csv(output_filename, index=False)

print(f"\nStress field data saved successfully to: {os.path.abspath(output_filename)}")

--- Stress Field Generation Configuration ---
Output File: stress.csv
Cube Size for Stress Field: 1.0e-03 m
Points per Dimension: 20
Total Stress Points: 8000
Maximum Stress Trace: 2000.0 MPa
--------------------------------------------

Generated Stress Points (first 5 rows):
     X    Y         Z         Trace
0  0.0  0.0  0.000000  0.000000e+00
1  0.0  0.0  0.000053  3.508772e+07
2  0.0  0.0  0.000105  7.017544e+07
3  0.0  0.0  0.000158  1.052632e+08
4  0.0  0.0  0.000211  1.403509e+08

Generated Stress Points (last 5 rows - 应该在 (L,L,L) 附近显示最大应力):
          X      Y         Z         Trace
7995  0.001  0.001  0.000789  1.859649e+09
7996  0.001  0.001  0.000842  1.894737e+09
7997  0.001  0.001  0.000895  1.929825e+09
7998  0.001  0.001  0.000947  1.964912e+09
7999  0.001  0.001  0.001000  2.000000e+09

Max Trace in generated data: 2.00e+09 Pa (2000.00 MPa)
Min Trace in generated data: 0.00e+00 Pa (0.00 MPa)

Stress field data saved successfully to: /Users/cool/Desktop/tensor/projects